# Fantasy points

In [27]:
import pandas as pd
import numpy as np

from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.static import teams

In [28]:
def match_A_B(team_id1, team_id2):
    final_df = pd.DataFrame()
    for i in range(0,4):
        season = f"202{i}-2{i+1}"
        abbreviation_A = teams.find_team_name_by_id(team_id1)["abbreviation"]
        abbreviation_B = teams.find_team_name_by_id(team_id2)["abbreviation"]
        
        # Ottieni i dettagli delle partite giocate nella stagione specificata
        game_log = leaguegamelog.LeagueGameLog(season=season)
        
        # Ottieni i dati dei DataFrame
        game_log_data = game_log.get_data_frames()
        
        # Stampa i dettagli delle partite giocate nella stagione specificata
        game_log_data = game_log_data[0][game_log_data[0]['TEAM_ID'] == team_id1]
        
        game_log_data_A_B = game_log_data[game_log_data['MATCHUP'].str.contains(abbreviation_B)]
        
        final_df = pd.concat([final_df, game_log_data_A_B], axis = 0)
    return final_df

In [29]:
def get_FP_player(player_id):

    career = playercareerstats.PlayerCareerStats(player_id=player_id)

    # pandas data frames (optional: pip install pandas)
    player_stats = career.get_data_frames()[0]

    # Calcola i fantasy points utilizzando le statistiche disponibili e i coefficienti appropriati
    coeff_pts = 1.0
    coeff_reb = 1.2
    coeff_ast = 1.5
    coeff_stl = 3.0
    coeff_blk = 3.0
    coeff_to = -1.0

    fantasy_points = (player_stats['PTS'] * coeff_pts +
                    player_stats['REB'] * coeff_reb +
                    player_stats['AST'] * coeff_ast +
                    player_stats['STL'] * coeff_stl +
                    player_stats['BLK'] * coeff_blk +
                    player_stats['TOV'] * coeff_to)
    ## ritorno solo il FP dell'ultima season
    return fantasy_points.iloc[-1]
    

In [30]:
def titolari(game_id):
    # Ottieni i dati della partita
    boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)

    # Ottieni i dati dei DataFrame
    boxscore_data = boxscore.get_data_frames()

    start_positions = boxscore_data[0][boxscore_data[0]["START_POSITION"].apply(lambda x: len(x) != 0)]
    
    return start_positions

In [31]:
##sto usando solo i player titolari, non tutta la squadra!! parlare di questo
def get_FP_team(team_players_id):
    fp_team = 0
    for player in team_players_id:
        fp_team += get_FP_player(player)
    return fp_team

In [32]:
# Definisci gli ID delle squadre (ad esempio, Los Angeles Lakers e Boston Celtics)
team_id1 = 1610612747  # Los Angeles Lakers
team_id2 = 1610612738  # Boston Celtics

final_df = match_A_B(team_id1=team_id1, team_id2=team_id2)

final_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
556,22020,1610612747,LAL,Los Angeles Lakers,0022000302,2021-01-30,LAL @ BOS,W,240,37,...,32,45,20,8,4,14,15,96,1,1
1649,22020,1610612747,LAL,Los Angeles Lakers,0022000842,2021-04-15,LAL vs. BOS,L,240,44,...,25,36,28,12,2,8,16,113,-8,1
461,22021,1610612747,LAL,Los Angeles Lakers,0022100232,2021-11-19,LAL @ BOS,L,240,39,...,26,33,20,5,6,12,29,108,-22,1
725,22021,1610612747,LAL,Los Angeles Lakers,0022100365,2021-12-07,LAL vs. BOS,W,240,48,...,38,51,25,7,4,16,21,117,15,1
816,22022,1610612747,LAL,Los Angeles Lakers,0022200413,2022-12-13,LAL vs. BOS,L,265,44,...,42,49,23,7,9,12,17,118,-4,1
1482,22022,1610612747,LAL,Los Angeles Lakers,0022200749,2023-01-28,LAL @ BOS,L,265,47,...,36,47,30,4,0,12,23,121,-4,1
864,22023,1610612747,LAL,Los Angeles Lakers,0022300403,2023-12-25,LAL vs. BOS,L,240,43,...,30,43,24,4,1,12,19,115,-11,1
1425,22023,1610612747,LAL,Los Angeles Lakers,0022300684,2024-02-01,LAL @ BOS,W,240,38,...,37,52,30,11,2,7,8,114,9,1


In [33]:
for index,row in final_df.iterrows():
    start_pos = titolari(row["GAME_ID"])
    #print(f"{type(row['GAME_ID'])} - {row['GAME_ID']}")
    print(f"Titolari game_id: {row['GAME_ID']} tra {row['MATCHUP']} del {row['GAME_DATE']}")
    
    team_players_id = start_pos[start_pos["TEAM_ABBREVIATION"] == row['MATCHUP'].split(" ")[0]]["PLAYER_ID"].to_list()
    FP = get_FP_team(team_players_id)
    print(f"Squadra: {row['MATCHUP'].split(' ')[0].strip()} FP: {FP}")
    print(start_pos[start_pos["TEAM_ABBREVIATION"] == row['MATCHUP'].split(" ")[0]]["PLAYER_NAME"].to_list())
    
    team_players_id = start_pos[start_pos["TEAM_ABBREVIATION"] == row['MATCHUP'].split(" ")[-1]]["PLAYER_ID"].to_list()
    FP = get_FP_team(team_players_id)
    print(f"Squadra: {row['MATCHUP'].split(' ')[-1]} FP: {FP}")
    print(start_pos[start_pos["TEAM_ABBREVIATION"] == row['MATCHUP'].split(" ")[-1]]["PLAYER_NAME"].to_list())
    print()

Titolari game_id: 0022000302 tra LAL @ BOS del 2021-01-30
Squadra: LAL FP: 10454.400000000001
['LeBron James', 'Anthony Davis', 'Marc Gasol', 'Kentavious Caldwell-Pope', 'Dennis Schroder']
Squadra: BOS FP: 6878.0
['Jaylen Brown', 'Jayson Tatum', 'Daniel Theis', 'Marcus Smart', 'Kemba Walker']

Titolari game_id: 0022000842 tra LAL vs. BOS del 2021-04-15
Squadra: LAL FP: 6719.799999999999
['Wesley Matthews', 'Kyle Kuzma', 'Marc Gasol', 'Kentavious Caldwell-Pope', 'Dennis Schroder']
Squadra: BOS FP: 6503.900000000001
['Jayson Tatum', 'Jaylen Brown', 'Tristan Thompson', 'Marcus Smart', 'Kemba Walker']

Titolari game_id: 0022100232 tra LAL @ BOS del 2021-11-19
Squadra: LAL FP: 9590.199999999999
['Talen Horton-Tucker', 'LeBron James', 'Anthony Davis', 'Avery Bradley', 'Russell Westbrook']
Squadra: BOS FP: 8090.5
['Jayson Tatum', 'Grant Williams', 'Al Horford', 'Dennis Schroder', 'Marcus Smart']

Titolari game_id: 0022100365 tra LAL vs. BOS del 2021-12-07
Squadra: LAL FP: 9590.199999999999
['

In [97]:
boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id="0022100232")

# Ottieni i dati dei DataFrame
boxscore_data = boxscore.get_data_frames()[0]

# Per ogni partita FP pesato

In [133]:
def ft_FPW_player(player, boxscore_data_cleaned):
    # Calcola i fantasy points utilizzando le statistiche disponibili e i coefficienti appropriati
    coeff_pts = 1.0
    coeff_reb = 1.2
    coeff_ast = 1.5
    coeff_stl = 3.0
    coeff_blk = 3.0
    coeff_to = -1.0
    
    player_stats=boxscore_data_cleaned[boxscore_data_cleaned["PLAYER_ID"] == player]

    fantasy_points = (player_stats['PTS'] * coeff_pts +
                    player_stats['REB'] * coeff_reb +
                    player_stats['AST'] * coeff_ast +
                    player_stats['STL'] * coeff_stl +
                    player_stats['BLK'] * coeff_blk +
                    player_stats['TO'] * coeff_to)
    # print(player)
    # print(fantasy_points)
    # print(player_stats['MIN'].to_list()[0])
    
    minutes = float(f"{player_stats['MIN'].to_list()[0].split('.')[0]}.{player_stats['MIN'].to_list()[0].split(':')[-1]}")
    #print(fantasy_points*minutes)
    ##pesato in base ai minuti giocati
    #print(f"minutes: {minutes}, fp: {float(fantasy_points)}")
    fantasy_points_w = float(fantasy_points) * minutes
    #print("fw: ", fantasy_points_w)
    ## ritorno solo il FP dell'ultima season
    return fantasy_points_w

In [134]:
def get_FP_team_weigthed(team_players_id, boxscore_data_cleaned):
    fp_team = 0

    for player in team_players_id:
        fp_team += ft_FPW_player(player, boxscore_data_cleaned)
    return fp_team

In [136]:
for index,row in final_df.iterrows():
    
    boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(row["GAME_ID"])

    # Ottieni i dati dei DataFrame
    boxscore_data_cleaned = boxscore.get_data_frames()[0].dropna()
    
    #print(f"{type(row['GAME_ID'])} - {row['GAME_ID']}")
    print(f"Titolari game_id: {row['GAME_ID']} tra {row['MATCHUP']} del {row['GAME_DATE']}")
    
    team_players_id = boxscore_data_cleaned[boxscore_data_cleaned["TEAM_ABBREVIATION"] == row['MATCHUP'].split(" ")[0]]["PLAYER_ID"].to_list()
    FP = get_FP_team_weigthed(team_players_id, boxscore_data_cleaned)
    print(f"Squadra: {row['MATCHUP'].split(' ')[0].strip()} FP: {FP}")
    print(boxscore_data_cleaned[boxscore_data_cleaned["TEAM_ABBREVIATION"] == row['MATCHUP'].split(" ")[0]]["PLAYER_NAME"].to_list())
    
    team_players_id = boxscore_data_cleaned[boxscore_data_cleaned["TEAM_ABBREVIATION"] == row['MATCHUP'].split(" ")[-1]]["PLAYER_ID"].to_list()
    FP = get_FP_team_weigthed(team_players_id, boxscore_data_cleaned)
    print(f"Squadra: {row['MATCHUP'].split(' ')[-1]} FP: {FP}")
    print(boxscore_data_cleaned[boxscore_data_cleaned["TEAM_ABBREVIATION"] == row['MATCHUP'].split(" ")[-1]]["PLAYER_NAME"].to_list())
    print()

Titolari game_id: 0022000302 tra LAL @ BOS del 2021-01-30
Squadra: LAL FP: 6282.4259999999995
['LeBron James', 'Anthony Davis', 'Marc Gasol', 'Kentavious Caldwell-Pope', 'Dennis Schroder', 'Kyle Kuzma', 'Talen Horton-Tucker', 'Montrezl Harrell', 'Alex Caruso']
Squadra: BOS FP: 6410.478999999999
['Jaylen Brown', 'Jayson Tatum', 'Daniel Theis', 'Marcus Smart', 'Kemba Walker', 'Robert Williams III', 'Jeff Teague', 'Tristan Thompson', 'Semi Ojeleye', 'Aaron Nesmith']



/var/folders/6m/jckxkf7x18q5443ply9521w00000gn/T/ipykernel_25077/761171805.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  fantasy_points_w = float(fantasy_points) * minutes


Titolari game_id: 0022000842 tra LAL vs. BOS del 2021-04-15
Squadra: LAL FP: 5202.423000000001
['Wesley Matthews', 'Kyle Kuzma', 'Marc Gasol', 'Kentavious Caldwell-Pope', 'Dennis Schroder', 'Talen Horton-Tucker', 'Montrezl Harrell', 'Alex Caruso', 'Ben McLemore', 'Alfonzo McKinnie', 'Devontae Cacok', 'Kostas Antetokounmpo']
Squadra: BOS FP: 5927.657999999999
['Jayson Tatum', 'Jaylen Brown', 'Tristan Thompson', 'Marcus Smart', 'Kemba Walker', 'Romeo Langford', 'Payton Pritchard', 'Luke Kornet', 'Grant Williams', 'Carsen Edwards', 'Moritz Wagner', 'Aaron Nesmith', 'Tremont Waters', 'Semi Ojeleye']

Titolari game_id: 0022100232 tra LAL @ BOS del 2021-11-19
Squadra: LAL FP: 5424.959000000002
['Talen Horton-Tucker', 'LeBron James', 'Anthony Davis', 'Avery Bradley', 'Russell Westbrook', 'Carmelo Anthony', 'Malik Monk', 'Rajon Rondo', 'Wayne Ellington', 'Dwight Howard', 'Kent Bazemore']
Squadra: BOS FP: 6989.777999999999
['Jayson Tatum', 'Grant Williams', 'Al Horford', 'Dennis Schroder', 'Mar